In [214]:
import requests
from bs4 import BeautifulSoup

URL = "https://www.oddsshark.com/tennis/atp/odds"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [215]:
#get odds for tennis players
odds_results = soup.find_all("div", {"class": "odds-moneyline moneyline-only"})
#Get tennis player names
name_results = soup.find_all(lambda tag: tag.name == 'div' and 
                                   tag.get('class') == ['participant-name'])

In [217]:
import pandas as pd
df = pd.DataFrame()

In [220]:
player_one_list = []
player_two_list = []

#split players between player one and player two
for index, row in enumerate(list_names):
    if index % 2 == 0:
        player_one_list.append(row)
    if index % 2 != 0:
        player_two_list.append(row)

#delete whitespace for player one
cleaned_names_one = []
for name in player_one_list:
    cleaned_name_one = name.strip()
    cleaned_names_one.append(cleaned_name_one)
    
#delete whitespace for player two
cleaned_names_two = []
for name in player_two_list:
    cleaned_name_two = name.strip()
    cleaned_names_two.append(cleaned_name_two)

In [221]:
#Add player one and player two the dataframe
df['player_one'] = cleaned_names_one
df['player_two'] = cleaned_names_two

#print the dataframe
df

,player_one,player_two
0,Novak Djokovic,Daniil Medvedev


In [223]:
#odds_results
list_odds = []

#add odds to a list
for i in odds_results:
    list_odds.append(i.text)

In [225]:
current_odd_list = [v for i, v in enumerate(list_odds) if i % 4 < 2]

player_one_list = []
player_two_list = []

#split odds between player one and player two
for index, row in enumerate(current_odd_list):
    if index % 2 == 0:
        player_one_list.append(row)
    if index % 2 != 0:
        player_two_list.append(row)

df['player_one_odds'] = player_one_list
df['player_two_odds'] = player_two_list

df

,player_one,player_two,player_one_odds,player_two_odds
0,Novak Djokovic,Daniil Medvedev,-250,+200


In [228]:
#Get tennis player seed from atp website (espn is blocked)

URL = "https://www.atptour.com/en/rankings/singles"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

In [229]:
# Creating list with all tables on website
tables = soup.find_all('table')

# Looking for the table with the classes 'mega-table'
table = soup.find('table', class_='mega-table')

In [230]:
import re

#Get ranking seed
player_ranking = []

for name in table.tbody.find_all("td", class_="rank-cell border-left-4 border-right-dash-1"):
    salary = name.parent.find_all('td')[-1]  # last cell in the row
    player_ranking.append(name.get_text())

cleaned_player_data = [row.replace('\n', '').replace('\r', '') for row in player_ranking]
#Remove the whitespace
cleaned_player_data = [re.sub(r'\s+', '', row) for row in cleaned_player_data]

In [231]:
#Get tennis player name
name_list = []
elements_with_class = soup.find_all('span', class_='player-cell-wrapper')

for element in elements_with_class:
    # Extract the text content of the <span> element
    link_text = element.text
    
    # Extract the href attribute of the <span> element (if present)
    href = element.get('href')

    # Append each name to the list
    name_list.append(link_text)

In [237]:
import re

cleaned_name_data = [row.replace('\n', '').replace('\r', '') for row in name_list]
#Remove the whitespace
cleaned_name_data = [re.sub(r'\s+', '', row) for row in cleaned_name_data]

# Function to insert spaces between capital letters
def insert_spaces(word):
    return re.sub(r'([A-Z])', r' \1', word)

# Apply the function to each word in the list
cleaned_name_data = [insert_spaces(word) for word in cleaned_name_data]

#delete whitespace
cleaned_names = []
for name in cleaned_name_data:
    cleaned_name = name.lstrip()
    cleaned_names.append(cleaned_name)

#print cleaned tennis names
#cleaned_names

In [232]:
import pandas as pd
df_seed = pd.DataFrame()

df_seed['player_one_seed'] = cleaned_player_data
df_seed['player_one'] = cleaned_names



In [234]:
# Merge the two DataFrames based on the 'key' column
merged_df = pd.merge(df, df_seed, on='player_one', how='inner')

In [238]:
df_seed = pd.DataFrame()

df_seed['player_two_seed'] = cleaned_player_data
df_seed['player_two'] = cleaned_names

#print out dataframe with seeds
df_seed

,player_two_seed,player_two
0,1,Carlos Alcaraz
1,2,Novak Djokovic
2,3,Daniil Medvedev
3,4,Holger Rune
4,5,Casper Ruud
...,...,...
95,96,David Goffin
96,97,Marc- Andrea Huesler
97,98,Pavel Kotov
98,99,Hugo Gaston


In [236]:
# Merge the two DataFrames based on the 'key' column
merged_df = pd.merge(merged_df, df_seed, on='player_two', how='inner')

merged_df

,player_one,player_two,player_one_odds,player_two_odds,player_one_seed,player_two_seed
0,Novak Djokovic,Daniil Medvedev,-250,+200,2,3
